In [ ]:
data_period = '202001to202309'

In [1]:
'''
東京用
'''
from bs4 import BeautifulSoup
import requests
from datetime import datetime  as dt, timedelta, datetime
import pandas as pd
from tqdm.notebook import tqdm


# 取得したデータの変換: 方角を整数値に変換、北を0°として360°で表記
def kanji_direction_to_degree(kanji_direction):
    direction = None

    match   kanji_direction:
        case '北北東':
            direction = 23
        case '東北東':
            direction = 68           
        case '東南東':
            direction = 113
        case '南南東':
            direction = 158
        case '南南西':
            direction = 203
        case '西南西':
            direction = 248
        case '西北西':
            direction = 293
        case '北北西':
            direction = 335
        case '北東':
            direction = 45
        case '南東':
            direction = 135
        case '南西':
            direction = 225
        case '北西':
            direction = 315
        case '北':
            direction = 0
        case '東':
            direction = 90
        case '南':
            direction = 180
        case '西':
            direction = 270
        case _:  # 判別不能なデータがきたとき
            direction = None
    return direction


# 取得したデータの変換:  float型に変換する。変換できない文字は0.0で代用。
# ※観測データのうち、データが無い（または変換不能なデータが当てはめられている）のを
#  測定結果が0とか測定限界以下だったんだな、って思って差し支えないデータはこっちを使う
#  例: 降雨量、降雪量
def str2float_z(str):
    try:
        return float(str)
    except:
        return 0.0

    
# 取得したデータの変換: float型に変換する。変換できない場合はNoneを返す。
# ※観測データのうち、データが無い（または変換不能なデータが当てはめられている）のを
#  0として考えると差し支えがあるデータの場合(つまり0と言うデータにも意味がある場合)は
# こっちを使う。
# #  例: 気温、気圧、湿度等
def str2float_n(str):
    try:
        return float(str)
    except:
        return None

    
# 辞書型のデータをdfに変換
def convert_dict_to_df(dic):
    weather_df = pd.DataFrame()
    for i in range(len(dic)):
        df = pd.DataFrame(dic[i].values(), index=dic[i].keys()).T
        weather_df = pd.concat([weather_df, df])
    return weather_df.reset_index(drop=True)


#気象庁のWEBサイトから指定日・指定場所の１時間毎の天気データを取得する
# 取得したデータは辞書形式の配列で返す
def fetch_daily_weather(date, prec_no, block_no):
    #date_f = dt.strptime(date, '%Y-%m-%d')

    # 日付データを取り込む
    year = date.year
    month = date.month
    day = date.day
    
    # 気象庁の時間毎のデータを取得するためのURL
    base_url="https://www.data.jma.go.jp/obd/stats/etrn/view/hourly_s1.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=%s&view=p1"


    try:
        # 指定URLからHTMLデータを取得する
        r = requests.get(base_url % (prec_no, block_no, year, month, day))
        r.encoding = r.apparent_encoding
        # 取得したデータをBeautifulSoupで解析する
        soup = BeautifulSoup(r.text, 'html.parser')
        rows = soup.findAll('tr', class_='mtx')
    except:
        return [] #空の配列を返す
    
    # 最初の２行は見出しなので削除する
    rows = rows[2:]

    # 時間毎の気象データは配列に格納。
    # 格納する配列を初期化する
    d_weather = []

    # 時間ごとに分割
    for row in rows:
        #項目毎に分割
        items = row.findAll('td')

        # 左側から数えて何番目のitemか、ということで場合分けして処理する

        # 時（いま何時！？？）
        hour = items[0].text
        if(hour == '24'):  #24時は、次の日の0時にする
            d = dt.strptime('%s-%s-%s' % (year, month, day), '%Y-%m-%d')
            d = d + timedelta(1)  # 次の日の日付
            date_time = dt.strptime('%s-%s-%s %s:0:0' % (d.year, d.month, d.day, '0'), '%Y-%m-%d %H:%M:%S')
        else:
            date_time = dt.strptime('%s-%s-%s %s:0:0' % (year, month, day, items[0].text), '%Y-%m-%d %H:%M:%S')

        # 気圧(地表)(hPa)
        pressure_ground = str2float_n(items[1].text)
        # 気圧(海面)(hPa)
        pressure_sea    = str2float_n(items[2].text)
        # 降水量(mm)
        rainfall = str2float_z(items[3].text)
        # 気温(℃)
        temperature = str2float_n(items[4].text)
        # 露点湿度(℃)
        dew_point_humidity = str2float_z(items[5].text)
        # 蒸気圧(hPa)
        vapor_pressure = str2float_n(items[6].text)
        # 湿度(%)
        humidity = str2float_n(items[7].text)
        # 風速(m/s)
        windspeed_value = str2float_n(items[8].text)
        # 風向
        windspeed_direction = kanji_direction_to_degree(items[9].text)
        # 日照時間(h)
        sunshine_hours = str2float_z(items[10].text)
        # 全天日射量(MJ/m2)
        global_solar_radiation = str2float_z(items[11].text)
        # 降雪(cm)
        snowfall = str2float_z(items[12].text)
        # 積雪(cm)
        snowdepth = str2float_z(items[13].text)

        # 天気
        weather = ''  #空欄だったら''をあてはめる
        img = items[14].img #imgタグを取得
        if(img):
            # データが入っていればalt属性を取得する
            weather =  img.attrs.get('alt', '')

        # 雲量
        if(len(items[15].text) > 0):
            cloud_amount = items[15].text
        else:
            # 空欄だったらNoneをあてはめる
            cloud_amount = None
        
        # 視程(km)
        visibility = str2float_n(items[16].text)

        # itemを処理し終わったので、辞書にまとめる
        h_weather = {   'date_time' : date_time,
                        'prec_no'   : prec_no,
                        'block_no'  : block_no,
                        'pressure_ground': pressure_ground, 
                        'pressure_sea': pressure_sea, 
                        'rainfall' : rainfall,
                        'temperature' : temperature,
                        'dew_point_humidity' : dew_point_humidity,
                        'vapor_pressure' : vapor_pressure,
                        'humidity' : humidity,
                        'windspeed_value' : windspeed_value,
                        'windspeed_direction' : windspeed_direction,
                        'sunshine_hours' : sunshine_hours,
                        'global_solar_radiation' : global_solar_radiation,
                        'snowfall' : snowfall,
                        'snowdepth' : snowdepth,
                        'weather' : weather,
                        'cloud_amount' : cloud_amount,
                        'visibility' : visibility
                         }
        # 辞書データを配列に追加する
        d_weather.append(h_weather)
    return convert_dict_to_df(d_weather)


# dfの1行目は午前1時からになってしまうので、欲しい日付の前日から取得
date = pd.date_range(start='2019/12/31', end='2023/9/30')
# d = [dt.strptime(dat, '%Y-%m-d')for dat in date]
output = pd.DataFrame()
for i in tqdm(date):
    '''
    Tokyo: prec_no=44, block_no=47662
    '''
    _df = fetch_daily_weather(date=i, prec_no='44', block_no='47662')
    output = pd.concat([output, _df])
output = output.reset_index(drop=True)

  0%|          | 0/1370 [00:00<?, ?it/s]

In [3]:
new_output = output.iloc[23:-1,:]
new_output

,date_time,prec_no,block_no,pressure_ground,pressure_sea,rainfall,temperature,dew_point_humidity,vapor_pressure,humidity,windspeed_value,windspeed_direction,sunshine_hours,global_solar_radiation,snowfall,snowdepth,weather,cloud_amount,visibility
23,2020-01-01 00:00:00,44,47662,1015.9,1018.9,0.0,3.7,-7.7,3.4,43.0,4.5,335,0.0,0.0,0.0,0.0,,None,None
24,2020-01-01 01:00:00,44,47662,1016.7,1019.7,0.0,3.7,-7.7,3.4,43.0,6.0,315,0.0,0.0,0.0,0.0,,None,None
25,2020-01-01 02:00:00,44,47662,1016.9,1019.9,0.0,3.5,-7.9,3.4,43.0,4.3,335,0.0,0.0,0.0,0.0,,None,None
26,2020-01-01 03:00:00,44,47662,1017.1,1020.1,0.0,3.6,-7.5,3.5,44.0,4.8,335,0.0,0.0,0.0,0.0,晴れ,4,20.0
27,2020-01-01 04:00:00,44,47662,1017.4,1020.4,0.0,4.0,-7.4,3.5,43.0,4.7,315,0.0,0.0,0.0,0.0,,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32874,2023-09-30 19:00:00,44,47662,1005.6,1008.4,0.0,25.5,21.8,26.1,80.0,2.7,180,0.0,0.0,0.0,0.0,,None,None
32875,2023-09-30 20:00:00,44,47662,1005.4,1008.2,0.0,25.2,22.1,26.6,83.0,3.5,180,0.0,0.0,0.0,0.0,,None,None
32876,2023-09-30 21:00:00,44,47662,1005.0,1007.8,0.0,25.9,20.5,24.1,72.0,4.0,203,0.0,0.0,0.0,0.0,晴れ,5,30.0
32877,2023-09-30 22:00:00,44,47662,1004.7,1007.5,0.0,26.0,20.8,24.5,73.0,5.5,203,0.0,0.0,0.0,0.0,,None,None


In [4]:
# new_output.to_csv('/Users/koki/PycharmProjects/MasterThesis/data/raw/weather_tokyo_' + data_period + '.csv', index=False, encoding='utf-8')

In [5]:
'''
東京以外用
'''
from bs4 import BeautifulSoup
import requests
from datetime import datetime  as dt, timedelta, datetime
import pandas as pd
from tqdm.notebook import tqdm


# 取得したデータの変換: 方角を整数値に変換、北を0°として360°で表記
def kanji_direction_to_degree(kanji_direction):
    direction = None

    match   kanji_direction:
        case '北北東':
            direction = 23
        case '東北東':
            direction = 68           
        case '東南東':
            direction = 113
        case '南南東':
            direction = 158
        case '南南西':
            direction = 203
        case '西南西':
            direction = 248
        case '西北西':
            direction = 293
        case '北北西':
            direction = 335
        case '北東':
            direction = 45
        case '南東':
            direction = 135
        case '南西':
            direction = 225
        case '北西':
            direction = 315
        case '北':
            direction = 0
        case '東':
            direction = 90
        case '南':
            direction = 180
        case '西':
            direction = 270
        case _:  # 判別不能なデータがきたとき
            direction = None
    return direction


# 取得したデータの変換:  float型に変換する。変換できない文字は0.0で代用。
# ※観測データのうち、データが無い（または変換不能なデータが当てはめられている）のは
#  測定結果が0とか測定限界以下だったんだな、って思って差し支えないデータはこっちを使う
#  例: 降雨量、降雪量
def str2float_z(str):
    try:
        return float(str)
    except:
        return 0.0

    
# 取得したデータの変換: float型に変換する。変換できない場合はNoneを返す。
# ※観測データのうち、データが無い（または変換不能なデータが当てはめられている）のは
#  0として考えると差し支えがあるデータの場合(つまり0と言うデータにも意味がある場合)は
# こっちを使う。
# #  例: 気温、気圧、湿度等
def str2float_n(str):
    try:
        return float(str)
    except:
        return None

    
# 辞書型のデータをdfに変換
def convert_dict_to_df(dic):
    weather_df = pd.DataFrame()
    for i in range(len(dic)):
        df = pd.DataFrame(dic[i].values(), index=dic[i].keys()).T
        weather_df = pd.concat([weather_df, df])
    return weather_df.reset_index(drop=True)


#気象庁のWEBサイトから指定日・指定場所の１時間毎の天気データを取得する
# 取得したデータは辞書形式の配列で返す
def fetch_daily_weather(date, prec_no, block_no):
    #date_f = dt.strptime(date, '%Y-%m-%d')

    # 日付データを取り込む
    year = date.year
    month = date.month
    day = date.day
    
    # 気象庁の時間毎のデータを取得するためのURL
    base_url="https://www.data.jma.go.jp/obd/stats/etrn/view/hourly_a1.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=%s&view=p1"


    try:
        # 指定URLからHTMLデータを取得する
        r = requests.get(base_url % (prec_no, block_no, year, month, day))
        r.encoding = r.apparent_encoding
        # 取得したデータをBeautifulSoupで解析する
        soup = BeautifulSoup(r.text, 'html.parser')
        rows = soup.findAll('tr', class_='mtx')
    except:
        return [] #空の配列を返す
    
    # 最初の２行は見出しなので削除する
    rows = rows[2:]

    # 時間毎の気象データは配列に格納。
    # 格納する配列を初期化する
    d_weather = []

    # 時間ごとに分割
    for row in rows:
        #項目毎に分割
        items = row.findAll('td')

        # 左側から数えて何番目のitemか、ということで場合分けして処理する

        # 時（いま何時！？？）
        hour = items[0].text
        if(hour == '24'):  #24時は、次の日の0時にする
            d = dt.strptime('%s-%s-%s' % (year, month, day), '%Y-%m-%d')
            d = d + timedelta(1)  # 次の日の日付
            date_time = dt.strptime('%s-%s-%s %s:0:0' % (d.year, d.month, d.day, '0'), '%Y-%m-%d %H:%M:%S')
        else:
            date_time = dt.strptime('%s-%s-%s %s:0:0' % (year, month, day, items[0].text), '%Y-%m-%d %H:%M:%S')

        # # 気圧(地表)(hPa)
        # pressure_ground = str2float_n(items[1].text)
        # # 気圧(海面)(hPa)
        # pressure_sea    = str2float_n(items[2].text)
        # 降水量(mm)
        rainfall = str2float_z(items[1].text)
        # 気温(℃)
        temperature = str2float_n(items[2].text)
        # # 露点湿度(℃)
        # dew_point_humidity = str2float_z(items[5].text)
        # # 蒸気圧(hPa)
        # vapor_pressure = str2float_n(items[6].text)
        # # 湿度(%)
        # humidity = str2float_n(items[7].text)
        # 風速(m/s)
        windspeed_value = str2float_n(items[6].text)
        # 風向
        windspeed_direction = kanji_direction_to_degree(items[7].text)
        # 日照時間(h)
        sunshine_hours = str2float_z(items[8].text)
        # # 全天日射量(MJ/m2)
        # global_solar_radiation = str2float_z(items[11].text)
        # # 降雪(cm)
        # snowfall = str2float_z(items[12].text)
        # # 積雪(cm)
        # snowdepth = str2float_z(items[13].text)

        # # 天気
        # weather = ''  #空欄だったら''をあてはめる
        # img = items[14].img #imgタグを取得
        # if(img):
        #     # データが入っていればalt属性を取得する
        #     weather =  img.attrs.get('alt', '')

#         # 雲量
#         if(len(items[15].text) > 0):
#             cloud_amount = items[15].text
#         else:
#             # 空欄だったらNoneをあてはめる
#             cloud_amount = None
        
#         # 視程(km)
#         visibility = str2float_n(items[16].text)

        # itemを処理し終わったので、辞書にまとめる
        h_weather = {   'date_time' : date_time,
                        'prec_no'   : prec_no,
                        'block_no'  : block_no,
                        # 'pressure_ground': pressure_ground, 
                        # 'pressure_sea': pressure_sea, 
                        'rainfall' : rainfall,
                        'temperature' : temperature,
                        # 'dew_point_humidity' : dew_point_humidity,
                        # 'vapor_pressure' : vapor_pressure,
                        # 'humidity' : humidity,
                        'windspeed_value' : windspeed_value,
                        'windspeed_direction' : windspeed_direction,
                        'sunshine_hours' : sunshine_hours,
                        # 'global_solar_radiation' : global_solar_radiation,
                        # 'snowfall' : snowfall,
                        # 'snowdepth' : snowdepth,
                        # 'weather' : weather,
                        # 'cloud_amount' : cloud_amount,
                        # 'visibility' : visibility
                         }
        # 辞書データを配列に追加する
        d_weather.append(h_weather)
    return convert_dict_to_df(d_weather)


# dfの1行目は午前1時からになってしますので、欲しい日付の前日から取得
date = pd.date_range(start='2019/12/31', end='2023/9/30')
# d = [dt.strptime(dat, '%Y-%m-d')for dat in date]
output = pd.DataFrame()
for i in tqdm(date):
    '''
    Hachioji: prec_no=44, block_no=0366
    '''
    _df = fetch_daily_weather(date=i, prec_no='44', block_no='0366')
    output = pd.concat([output, _df])
output = output.reset_index(drop=True)

  0%|          | 0/1370 [00:00<?, ?it/s]

In [6]:
new_output = output.iloc[23:-1,:]
new_output

,date_time,prec_no,block_no,rainfall,temperature,windspeed_value,windspeed_direction,sunshine_hours
23,2020-01-01 00:00:00,44,0366,0.0,2.2,1.7,335,0.0
24,2020-01-01 01:00:00,44,0366,0.0,0.6,2.7,293,0.0
25,2020-01-01 02:00:00,44,0366,0.0,-0.2,1.2,293,0.0
26,2020-01-01 03:00:00,44,0366,0.0,-0.7,0.4,248,0.0
27,2020-01-01 04:00:00,44,0366,0.0,-1.1,1.1,270,0.0
...,...,...,...,...,...,...,...,...
32874,2023-09-30 19:00:00,44,0366,0.0,24.6,1.4,180,0.0
32875,2023-09-30 20:00:00,44,0366,0.0,23.6,1.2,203,0.0
32876,2023-09-30 21:00:00,44,0366,0.0,22.9,1.0,225,0.0
32877,2023-09-30 22:00:00,44,0366,0.0,22.6,0.5,248,0.0


In [7]:
# new_output.to_csv('/Users/koki/PycharmProjects/MasterThesis/data/raw/weather_hachioji_' + data_period + '.csv', index=False, encoding='utf-8')